In [39]:
from __future__ import division
import operator
import nltk
import string
import numpy as np
import pandas as pd
import math
from math import log10
import copy

In [2]:
def isPunct(word):
    return len(word) == 1 and word in string.punctuation

def isNumeric(word):
    try:
        float(word) if '.' in word else int(word)
        return True
    except ValueError:
        return False

In [3]:
class RakeKeywordExtractor:

    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words())
        self.top_fraction = 1 # consider top third candidate keywords by score

    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            words = map(lambda x: "|" if x in self.stopwords else x,nltk.word_tokenize(sentence.lower()))
            ##words => NONSTOPWORD | NONSTOPWORD NONSTOPWORD NONSTOPWORD | | NONSTOPWORD | NONSTOPWORD NONSTOPWORD |
            phrase = []
            for word in words:
                if word == "|" or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)#Got At least 1 NonStopWord
                    phrase = []#Prepare for Next Continous NonStopWords
                else:
                    phrase.append(word)#NonStopWord
        return phrase_list

    def _calculate_word_scores(self, phrase_list):
        word_freq = nltk.FreqDist()
        word_degree = nltk.FreqDist()
        for phrase in phrase_list:
            degree = len(list(filter(lambda x: not isNumeric(x), phrase))) #Number of Distinct Words in the Phrase
            for word in phrase:
                word_freq[word] += 1
                word_degree[word]+=degree
        # word score = deg(w) / freq(w)
        word_scores = {}
        for word in word_freq.keys():
            word_scores[word] = word_degree[word] / word_freq[word]
        return word_scores

    def _calculate_phrase_scores(self, phrase_list, word_scores):
        phrase_scores = {}
        for phrase in phrase_list:
            phrase_score = 0
            for word in phrase:
                phrase_score += word_scores[word]
                phrase_scores[" ".join(phrase)] = phrase_score
        return phrase_scores
    
    def extract(self, text, incl_scores=False):
        sentences = nltk.sent_tokenize(text)
        phrase_list = self._generate_candidate_keywords(sentences)
        word_scores = self._calculate_word_scores(phrase_list)
        phrase_scores = self._calculate_phrase_scores(phrase_list, word_scores)
        sorted_phrase_scores = sorted(phrase_scores.items(),
        key=operator.itemgetter(1), reverse=True)
        n_phrases = len(sorted_phrase_scores)
        if incl_scores:
            return sorted_phrase_scores[0:int(n_phrases/self.top_fraction)]
        else:
            return map(lambda x: x[0],sorted_phrase_scores[0:int(n_phrases/self.top_fraction)])

In [4]:
def test(feedback):
    rake = RakeKeywordExtractor()
    keywords = rake.extract(feedback, incl_scores=True)
    return keywords
f1="""
    Compatibility of systems of linear constraints over the set of natural 
    numbers. Criteria of compatibility of a system of linear Diophantine 
    equations, strict inequations, and nonstrict inequations are considered. 
    Upper bounds for components of a minimal set of solutions and algorithms 
    of construction of minimal generating sets of solutions for all types of 
    systems are given. These criteria and the corresponding algorithms for 
    constructing a minimal supporting set of solutions can be used in solving 
    all the considered types of systems and systems of mixed types.
    """ 
test(f1)

[('minimal generating sets', 8.666666666666666),
 ('linear diophantine equations', 8.5),
 ('minimal supporting set', 7.666666666666666),
 ('minimal set', 4.666666666666666),
 ('linear constraints', 4.5),
 ('strict inequations', 4.0),
 ('nonstrict inequations', 4.0),
 ('natural numbers', 4.0),
 ('upper bounds', 4.0),
 ('mixed types', 3.666666666666667),
 ('corresponding algorithms', 3.5),
 ('considered types', 3.166666666666667),
 ('set', 2.0),
 ('types', 1.6666666666666667),
 ('algorithms', 1.5),
 ('considered', 1.5),
 ('compatibility', 1.0),
 ('given', 1.0),
 ('criteria', 1.0),
 ('construction', 1.0),
 ('used', 1.0),
 ('components', 1.0),
 ('system', 1.0),
 ('constructing', 1.0),
 ('solutions', 1.0),
 ('systems', 1.0),
 ('solving', 1.0)]

In [5]:
import json
from pprint import pprint
def parse(filename):
    with open(filename) as data_file:
        data = json.load(data_file) 
        return data

In [6]:
parsedData=parse("Musical_Instruments_5.json")

In [7]:
commentText=[]
rakeTags=[]
amazonMusicReviewDF=pd.DataFrame(columns=["ReviewText","RakePhrases"])
for i in np.arange(10):
    amazonMusicReviewDF.loc[i]=[parsedData[i]["reviewText"],test(parsedData[i]["reviewText"])]

In [8]:
amazonMusicReviewDF.loc[3].ReviewText,amazonMusicReviewDF.loc[3].RakePhrases

('Nice windscreen protects my MXL mic and prevents pops. Only thing is that the gooseneck is only marginally able to hold the screen in position and requires careful positioning of the clamp to avoid sagging.',
 [('nice windscreen protects', 9.0),
  ('requires careful positioning', 9.0),
  ('mxl mic', 4.0),
  ('avoid sagging', 4.0),
  ('marginally able', 4.0),
  ('prevents pops', 4.0),
  ('screen', 1.0),
  ('thing', 1.0),
  ('hold', 1.0),
  ('clamp', 1.0),
  ('position', 1.0),
  ('gooseneck', 1.0)])

In [9]:
i=4
amazonMusicReviewDF.loc[i].ReviewText,amazonMusicReviewDF.loc[i].RakePhrases

("This pop filter is great. It looks and performs like a studio filter. If you're recording vocals this will eliminate the pops that gets recorded when you sing.",
 [("'re recording vocals", 9.0),
  ('gets recorded', 4.0),
  ('studio filter', 4.0),
  ('pop filter', 4.0),
  ('performs like', 4.0),
  ('eliminate', 1.0),
  ('sing', 1.0),
  ('great', 1.0),
  ('looks', 1.0),
  ('pops', 1.0)])

In [10]:
i=7
amazonMusicReviewDF.loc[i].ReviewText,amazonMusicReviewDF.loc[i].RakePhrases

("I now use this cable to run from the output of my pedal chain to the input of my Fender Amp. After I bought Monster Cable to hook up my pedal board I thought I would try another one and update my guitar. I had been using a high end Planet Waves cable that I bought in the 1980's... Once I found out the input jacks on the new Monster cable didn't fit into the Fender Strat jack I was a little disappointed... I didn't return it and as stated I use it for the output on the pedal board. Save your money... I went back to my Planet Waves Cable...I payed $30.00 back in the eighties for the Planet Waves which now comes in at around $50.00. What I'm getting at is you get what you pay for. I thought Waves was a lot of money back in the day...but I haven't bought a guitar cable since this one...20 plus years and still working...Planet Waves wins.",
 [('still working ... planet waves wins', 28.75),
  ('one ... 20 plus years', 19.6),
  ('would try another one', 16.0),
  ('planet waves cable ...', 1

In [81]:
class RakeWithPMIKeywordExtractor:
    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words())
        self.top_fraction = 1 # consider top third candidate keywords by score
    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            words = map(lambda x: "|" if x in self.stopwords else x,nltk.word_tokenize(sentence.lower()))
            ##words => NONSTOPWORD | NONSTOPWORD NONSTOPWORD NONSTOPWORD | | NONSTOPWORD | NONSTOPWORD NONSTOPWORD |
            phrase = []
            for word in words:
                if word == "|" or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)#Got At least 1 NonStopWord
                    phrase = []#Prepare for Next Continous NonStopWords
                else:
                    phrase.append(word)#NonStopWord
        return phrase_list

    def _chkAndAddDictEntry(self,dictObj,key,value=1):
        if dictObj.get(key)==None:
            dictObj[key]=value
        else:
            dictObj[key]+=value
    def _chkAndInitiateDictEntry(self,dictOfDictObj,key,initialDict):
        if dictOfDictObj.get(key)==None:
            dictOfDictObj[key]=initialDict;
        return dictOfDictObj[key]
    #This implementation of PMI does ignore reapeating words in a KeyPhrase
    def _prepareJoinFreqMatrix(self,all_phrase_lists):
        corpusLevelDict={}#It is Symmetric corpusLevelDict["w1"]["w2"]=corpusLevelDict["w2"]["w1"]
        corpusLevelWordC={}
        wordLevelDict={}
        self.phraseCorpus=all_phrase_lists
        for phrase_list_all in all_phrase_lists:
            phrase_list=set(phrase_list_all)
            for word in phrase_list:
                wordLevelDict=self._chkAndInitiateDictEntry(corpusLevelDict,word,{})
                self._chkAndAddDictEntry(corpusLevelWordC,word)
                for neighWord in phrase_list:
                    #if word==neighWord:
                    #    continue
                    self._chkAndAddDictEntry(wordLevelDict,neighWord)
        self.JointFreq=corpusLevelDict;
        self.PMIMatrix=copy.deepcopy(self.JointFreq)
        self.wordFreqInKEAPhrases=corpusLevelWordC
        self.UniFreq=copy.deepcopy(corpusLevelWordC)
        return corpusLevelDict
    def _updatePMIDMatrix(self):
        wordFreqDict=self.wordFreqInKEAPhrases
        joinFreqDist=self.PMIMatrix
        allPhrases=self.phraseCorpus
        keyPhrasesC=len(allPhrases)
        wordFreqDict.update({k:v/keyPhrasesC for k, v in wordFreqDict.items()})
        for word in joinFreqDist.keys():
            joinFreqDist[word].update({k:v/keyPhrasesC for k, v in joinFreqDist[word].items()})
    def _getPMIValue(self,word1,word2):
        #Handle Here Unseen Data: With Some Smoothing: Say Laplace
        #MI(word1,word2)=p(word1,word2)log(p(word1,word2)/p(word1)p(word2))
        prob_word1_word2=self.PMIMatrix[word1][word2]
        prob_word1=self.wordFreqInKEAPhrases[word1]
        prob_word2=self.wordFreqInKEAPhrases[word2]
        return prob_word1_word2*log10(prob_word1_word2/(prob_word1*prob_word2))
    def _populatePMIValue(self):
        joinFreqDist=self.PMIMatrix
        for word in joinFreqDist.keys():
                joinFreqDist[word].update({k:self._getPMIValue(word,k) for k, v in joinFreqDist[word].items()})
    def _computePMIAndRankKeyPhrases(self):
        rankedDict={}
        for phrase in self.phraseCorpus:
            phraseStr=""
            ans=0.0
            for i in np.arange(len(phrase)):
                phraseStr=phraseStr+" "+phrase[i]
                j=i+1
                while j<len(phrase):
                    ans+=self.PMIMatrix[phrase[i]][phrase[j]]
                    j=j+1
            rankedDict[phraseStr]=ans
        rankedDict = sorted(rankedDict.items(),key=operator.itemgetter(1), reverse=True)
        self.rankedKeyPhrasesDict=rankedDict

In [82]:
def getListOfPharases(reviewDF,fieldName):
    listOfPhrases=[]
    rakePhrases=reviewDF[fieldName]
    for i in np.arange(len(rakePhrases)):
        rakePhrase=rakePhrases[i]
        for j in np.arange(len(rakePhrase)):
            listOfPhrases.append(rakePhrase[j][0].split(" "))
    return listOfPhrases
def testPMI():
    listOfPhrases=getListOfPharases(amazonMusicReviewDF,"RakePhrases")
    rake = RakeWithPMIKeywordExtractor()
    rake._prepareJoinFreqMatrix(listOfPhrases)
    return rake

In [83]:
rakeObj=testPMI()
rakeObj._updatePMIDMatrix()
rakeObj._populatePMIValue()
rakeObj._computePMIAndRankKeyPhrases()

In [84]:
rakeObj.rankedKeyPhrasesDict

[(' still working ... planet waves wins', 0.16546514496818118),
 (' lifetime warranty doesnt hurt either', 0.12075945297641516),
 (' planet waves cable ...', 0.08853783196089851),
 (' one ... 20 plus years', 0.0796868771529823),
 (' mike stand secure enough', 0.07289331357871119),
 (' metal clamp mount attaches', 0.06806395535950294),
 (' would try another one', 0.06034832227754551),
 (' planet waves cable', 0.058119293180434774),
 (' lowest prices pop filters', 0.056850532830048095),
 (' double cloth filter blocks', 0.056850532830048095),
 (' pop filter next', 0.03849631958370766),
 (' goose neck needs', 0.036446656789355596),
 (' old grape candy', 0.036446656789355596),
 (' requires careful positioning', 0.036446656789355596),
 (' nice windscreen protects', 0.036446656789355596),
 (' 90 degree plug', 0.036446656789355596),
 (' epiphone sheraton ii', 0.036446656789355596),
 (' learned last night', 0.036446656789355596),
 (' cant go wrong', 0.036446656789355596),
 (' still works great'

In [59]:
listOfPhrases=getListOfPharases(amazonMusicReviewDF,"RakePhrases")
listOfPhrases

[['lowest', 'prices', 'pop', 'filters'],
 ['pop', 'sounds'],
 ['well', 'buy'],
 ['honestly', 'work'],
 ["'s", 'supposed'],
 ['filters'],
 ['much'],
 ['recordings'],
 ['pricing'],
 ['might'],
 ['despite'],
 ['crisp'],
 ['one'],
 ['exactly'],
 ['write'],
 ['amazon'],
 ['old', 'grape', 'candy'],
 ['reminiscent', "'s", 'sake'],
 ['pleasing', 'aroma', 'like'],
 ['pop', 'filter', 'next'],
 ['pop', 'filter'],
 ['quite', 'affordable.i'],
 ['screens', 'carries'],
 ['added', 'bonus'],
 ['expensive', 'ones'],
 ['even', 'better'],
 ['may', 'even'],
 ['double', 'screened'],
 ['small', 'hint'],
 ['stop', 'putting'],
 ['smell'],
 ['dif'],
 ['one'],
 ['smelling'],
 ['well'],
 ['exactly'],
 ['product'],
 ['realized'],
 ['recording'],
 ['needed'],
 ['work'],
 ['used'],
 ['expected.as'],
 ['buy'],
 ['nose'],
 ['arrived'],
 ['metal', 'clamp', 'mount', 'attaches'],
 ['mike', 'stand', 'secure', 'enough'],
 ['double', 'cloth', 'filter', 'blocks'],
 ['goose', 'neck', 'needs'],
 ['would', 'otherwise', 'produce

In [29]:
print(rakeObj.PMIMatrix["pop"])
rakeObj._getPMIValue("pop","filter")

{'filters': 0.0053475935828877, 'prices': 0.0053475935828877, 'filter': 0.016042780748663103, 'next': 0.0053475935828877, 'lowest': 0.0053475935828877, 'sounds': 0.0053475935828877}


0.021674163410440494

In [141]:
len(listOfPhrases)

187

In [126]:
rakeObj=testPMI()

Corpus Dictionary Size  239
Total KeyPhrases  187


1

In [129]:
topLevelDict=rakeObj.PMIMatrix
print(topLevelDict["lowest"])
print(topLevelDict["pop"])
print(topLevelDict["filters"])
print(topLevelDict["filter"])

{'prices': 1, 'filters': 1, 'pop': 1}
{'filters': 1, 'sounds': 1, 'prices': 1, 'filter': 3, 'lowest': 1, 'next': 1}
{'prices': 1, 'lowest': 1, 'pop': 1}
{'studio': 1, 'pop': 3, 'cloth': 1, 'double': 1, 'blocks': 1, 'next': 1}


In [ ]:
listOfPhrases

In [ ]:
corpusLevelDict

In [35]:
p1=[["ab","w2","w3"],["cd","w1","w2","cd"],[]]
for phrase_list_all in p1:
    phrase_list=set(phrase_list_all)
    print(phrase_list)

{'w2', 'ab', 'w3'}
{'w1', 'cd', 'w2'}
set()


In [20]:
topLevelDict

{'hj': 156, 'nota': {'notb': 45.1435}, 'pyth': {}}

In [24]:
len(topLevelDict.keys())

3

In [28]:
amazonMusicReviewDF.to_excel("AmazonMusicReview_RakeKeyPharses.xls")